In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df =  pd.read_csv('Churn_Modelling.csv')

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
df = df.iloc[:, 3:]

# Feature Engineering

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder()

df['Gender'] = label_encoder.fit_transform(df['Gender'])

values = onehot_encoder.fit_transform(df[['Geography']])

onehot_df = pd.DataFrame(values.toarray(), columns=onehot_encoder.get_feature_names_out())

In [6]:
df = pd.concat([df, onehot_df], axis = 1)

In [7]:
df.drop(columns=['Geography'], inplace=True)

In [8]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [9]:
with open('label_encoding_gender.pkl', 'wb') as file:

    pickle.dump(label_encoder, file)


with open('onehot_encoding_geo.pkl', 'wb') as file:

    pickle.dump(onehot_encoder, file)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Exited']), df['Exited'], test_size=0.2, random_state=42)

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

with open('scaler.pkl', 'wb') as file:

    pickle.dump(scaler, file)



# ANN Implementation

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

2025-05-16 19:43:01.631736: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 19:43:01.633832: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 19:43:01.669958: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-16 19:43:01.669996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-16 19:43:01.671012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [13]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1], )), ## HL1 connected with inouut layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid')
])

2025-05-16 19:43:03.484236: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
import tensorflow

opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

# Compile the model

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [16]:
# Setup the TensorBoard

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback=TensorBoard(log_dir, histogram_freq=1)

In [17]:
# Setup the Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [18]:
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test),epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4063 - accuracy: 0.8324 - val_loss: 0.3508 - val_accuracy: 0.8580
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3537 - accuracy: 0.8568 - val_loss: 0.3572 - val_accuracy: 0.8535
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3478 - accuracy: 0.8575 - val_loss: 0.3489 - val_accuracy: 0.8585
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3451 - accuracy: 0.8565 - val_loss: 0.3417 - val_accuracy: 0.8650
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3400 - accuracy: 0.8615 - val_loss: 0.3479 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3381 - accuracy: 0.8605 - val_loss: 0.3444 - val_accuracy: 0.8590
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3389 - accuracy: 0.8618 - val_loss: 0.3337 - val_accuracy: 0.8640

In [19]:
model.save('model.h5')

In [20]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir logs/fit20250516-194303

Reusing TensorBoard on port 6007 (pid 47137), started 0:00:02 ago. (Use '!kill 47137' to kill it.)